# Cybersecurity Applications

> RBE applications for network security, threat detection, and anomaly monitoring

In [ ]:
#| default_exp rbe.cybersecurity

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from typing import Dict, List, Optional, Tuple, Callable
from fastcore.test import test_eq, test_close
from fastcore.all import *
from technical_blog.rbe.probability import normalize, entropy
from technical_blog.rbe.bayes_core import update
from technical_blog.rbe.particle_filter import run as pf_run
from technical_blog.rbe.estimator import estimate

## Threat Detection

In [ ]:
#| export
def intrusion_detection(evidence_sequence, prior_attack=0.05, verbose=False):
    "Sequential intrusion detection using Bayesian updating"
    # Evidence types and their likelihoods [P(evidence|attack), P(evidence|normal)]
    evidence_map = {
        'port_scan': [0.8, 0.1],
        'failed_login': [0.7, 0.3],
        'data_transfer': [0.6, 0.4],
        'normal_traffic': [0.1, 0.9],
        'privilege_escalation': [0.95, 0.01],
        'suspicious_process': [0.85, 0.05]
    }
    
    belief = np.array([prior_attack, 1 - prior_attack])
    history = [belief.copy()]
    
    for i, evidence in enumerate(evidence_sequence):
        if evidence in evidence_map:
            likelihood = np.array(evidence_map[evidence])
        else:
            likelihood = np.array([0.5, 0.5])  # Unknown evidence
        
        belief = update(belief, likelihood)
        history.append(belief.copy())
        
        if verbose:
            print(f"Step {i+1}: {evidence} -> P(attack)={belief[0]:.3f}")
    
    return {
        'final_belief': belief,
        'history': np.array(history),
        'attack_probability': belief[0]
    }

In [ ]:
# Test intrusion detection
evidence = ['normal_traffic', 'port_scan', 'failed_login', 'privilege_escalation']
result = intrusion_detection(evidence, verbose=True)
assert result['attack_probability'] > 0.9  # Should be very confident about attack

## Multi-Stage Attack Detection

In [ ]:
#| export
def multi_stage_detector(event_sequence, attack_patterns, threshold=0.7):
    "Detect multi-stage attack patterns"
    # Initialize beliefs for each pattern
    pattern_beliefs = {}
    for pattern_name in attack_patterns:
        pattern_beliefs[pattern_name] = np.array([0.8, 0.2])  # [benign, attack]
    
    results = {
        'detections': [],
        'pattern_probabilities': {name: [] for name in attack_patterns},
        'alerts': []
    }
    
    for t, event in enumerate(event_sequence):
        alerts_this_step = []
        
        for pattern_name, pattern_steps in attack_patterns.items():
            if event in pattern_steps:
                # Strong evidence for attack
                likelihood = np.array([0.1, 0.9])
            else:
                # Evidence against attack
                likelihood = np.array([0.8, 0.2])
            
            pattern_beliefs[pattern_name] = update(pattern_beliefs[pattern_name], likelihood)
            attack_prob = pattern_beliefs[pattern_name][1]
            
            results['pattern_probabilities'][pattern_name].append(attack_prob)
            
            if attack_prob > threshold:
                alerts_this_step.append(pattern_name)
        
        if alerts_this_step:
            results['alerts'].append((t, alerts_this_step))
            results['detections'].extend(alerts_this_step)
    
    return results

In [ ]:
# Test multi-stage detection
patterns = {
    'Reconnaissance': ['port_scan', 'dns_lookup', 'service_enum'],
    'Exploitation': ['exploit_attempt', 'payload_delivery', 'code_execution'],
    'Persistence': ['registry_modification', 'service_creation', 'scheduled_task']
}

events = ['port_scan', 'normal', 'dns_lookup', 'exploit_attempt', 'service_enum']
result = multi_stage_detector(events, patterns)

assert 'Reconnaissance' in result['detections']
print(f"Detected patterns: {set(result['detections'])}")

## Anomaly Detection with Particle Filters

In [ ]:
#| export
def network_anomaly_detector(traffic_data, baseline_mean, baseline_std, 
                           n_particles=500, anomaly_threshold=3.0):
    "Detect network anomalies using particle filtering"
    
    def transition(state, rng):
        # State = [traffic_level, anomaly_score]
        level, score = state
        
        # Traffic level evolves with some noise
        new_level = level + rng.normal(0, baseline_std * 0.1)
        
        # Anomaly score decays but can spike
        new_score = score * 0.9 + rng.exponential(0.1)
        
        return np.array([new_level, new_score])
    
    def likelihood(state, observation):
        level, score = state
        
        # How well does state explain observation?
        expected = baseline_mean + score * baseline_std
        diff = abs(observation - expected)
        
        return np.exp(-0.5 * (diff / baseline_std)**2)
    
    # Initialize around baseline
    def init_fn(n, dim, rng):
        levels = rng.normal(baseline_mean, baseline_std * 0.5, n)
        scores = rng.exponential(0.5, n)
        return np.column_stack([levels, scores])
    
    # Run particle filter
    result = pf_run(traffic_data, transition, likelihood, 
                   n_particles=n_particles, init_fn=init_fn)
    
    # Extract anomaly scores
    anomaly_scores = result['estimates'][:, 1]
    anomalies = anomaly_scores > anomaly_threshold
    
    return {
        'anomaly_scores': anomaly_scores,
        'anomalies': anomalies,
        'anomaly_times': np.where(anomalies)[0],
        'pf_result': result
    }

In [ ]:
# Test anomaly detection
rng = np.random.default_rng(42)

# Generate traffic data with anomalies
normal_traffic = rng.normal(100, 10, 50)
anomaly_traffic = rng.normal(200, 20, 10)  # Spike
traffic = np.concatenate([normal_traffic[:20], anomaly_traffic, normal_traffic[20:]])

result = network_anomaly_detector(traffic, baseline_mean=100, baseline_std=10)
print(f"Anomalies detected at times: {result['anomaly_times']}")
assert len(result['anomaly_times']) > 0  # Should detect some anomalies

## Threat Intelligence Fusion

In [ ]:
#| export
def fuse_threat_intel(sources, reliabilities, prior=None):
    "Fuse multiple threat intelligence sources"
    if prior is None:
        prior = np.array([0.1, 0.9])  # [threat, no_threat]
    
    belief = prior.copy()
    fusion_history = [belief.copy()]
    
    for source, reliability in zip(sources, reliabilities):
        # Source report: threat_detected (True/False)
        if source['threat_detected']:
            # Likelihood depends on source reliability
            likelihood = np.array([reliability, 1 - reliability])
        else:
            likelihood = np.array([1 - reliability, reliability])
        
        belief = update(belief, likelihood)
        fusion_history.append(belief.copy())
    
    # Calculate confidence
    confidence = 1 - entropy(belief)
    
    return {
        'final_assessment': belief,
        'threat_probability': belief[0],
        'confidence': confidence,
        'history': np.array(fusion_history)
    }

In [ ]:
# Test threat intelligence fusion
sources = [
    {'name': 'Commercial_TI', 'threat_detected': True},
    {'name': 'OSINT', 'threat_detected': True},
    {'name': 'Internal_SOC', 'threat_detected': False},
    {'name': 'Government_Feed', 'threat_detected': True}
]
reliabilities = [0.85, 0.70, 0.90, 0.95]

result = fuse_threat_intel(sources, reliabilities)
print(f"Threat probability: {result['threat_probability']:.3f}")
print(f"Confidence: {result['confidence']:.3f}")
assert result['threat_probability'] > 0.5  # Should indicate threat

## Adaptive Baseline Monitoring

In [ ]:
#| export
def adaptive_monitor(data, window_size=20, adapt_rate=0.1, 
                    alert_threshold=2.5, decay_rate=0.05):
    "Monitor with adaptive baseline and threat detection"
    
    results = {
        'threat_scores': [],
        'baselines': [],
        'alerts': [],
        'threat_belief': []
    }
    
    # Initialize
    if len(data) < window_size:
        baseline = np.mean(data)
        baseline_std = np.std(data) if len(data) > 1 else 1.0
    else:
        baseline = np.mean(data[:window_size])
        baseline_std = np.std(data[:window_size])
    
    threat_belief = np.array([0.5, 0.5])  # [normal, threat]
    
    for t, obs in enumerate(data):
        # Calculate deviation
        z_score = abs(obs - baseline) / (baseline_std + 1e-6)
        
        # Update threat belief
        if z_score < 1.0:
            likelihood = np.array([0.9, 0.1])
        elif z_score < 2.0:
            likelihood = np.array([0.7, 0.3])
        elif z_score < 3.0:
            likelihood = np.array([0.3, 0.7])
        else:
            likelihood = np.array([0.1, 0.9])
        
        threat_belief = update(threat_belief, likelihood)
        
        # Decay toward neutral during normal periods
        if z_score < 1.5:
            threat_belief[1] = threat_belief[1] * (1 - decay_rate) + 0.5 * decay_rate
            threat_belief[0] = 1 - threat_belief[1]
        
        # Alert if threat probability high
        threat_score = threat_belief[1]
        alert = z_score > alert_threshold and threat_score > 0.6
        
        results['threat_scores'].append(threat_score)
        results['baselines'].append(baseline)
        results['alerts'].append(alert)
        results['threat_belief'].append(threat_belief.copy())
        
        # Update baseline (adaptive)
        if z_score < 2.0:  # Only adapt during normal periods
            baseline = (1 - adapt_rate) * baseline + adapt_rate * obs
            # Update std using exponential moving average
            if t >= window_size:
                recent_data = data[max(0, t-window_size+1):t+1]
                baseline_std = (1 - adapt_rate) * baseline_std + adapt_rate * np.std(recent_data)
    
    return results

In [ ]:
# Test adaptive monitoring
# Generate data with drift and anomalies
t = np.arange(100)
baseline_drift = 100 + 0.1 * t  # Slow drift
noise = rng.normal(0, 5, 100)
data = baseline_drift + noise

# Add anomalies
anomaly_indices = [30, 31, 60, 61, 62]
for idx in anomaly_indices:
    data[idx] += rng.normal(30, 5)

result = adaptive_monitor(data)
alert_times = np.where(result['alerts'])[0]
print(f"Alerts at times: {alert_times}")
print(f"Total alerts: {sum(result['alerts'])}")

# Should detect most anomalies
assert len(alert_times) > 0

## Security Event Correlation

In [ ]:
#| export
def correlate_events(events, time_window=60, correlation_threshold=0.7):
    "Correlate security events to identify coordinated attacks"
    
    # Group events by time window
    event_groups = []
    current_group = []
    
    for event in events:
        if not current_group or event['timestamp'] - current_group[0]['timestamp'] <= time_window:
            current_group.append(event)
        else:
            if current_group:
                event_groups.append(current_group)
            current_group = [event]
    
    if current_group:
        event_groups.append(current_group)
    
    # Analyze each group
    correlations = []
    
    for group in event_groups:
        if len(group) < 2:
            continue
        
        # Calculate correlation score based on:
        # 1. Temporal proximity
        # 2. Source/target similarity
        # 3. Attack pattern matching
        
        time_span = group[-1]['timestamp'] - group[0]['timestamp']
        time_score = 1.0 - (time_span / time_window)
        
        # Check for common sources or targets
        sources = [e.get('source', '') for e in group]
        targets = [e.get('target', '') for e in group]
        
        source_overlap = len(set(sources)) / len(sources) if sources else 0
        target_overlap = len(set(targets)) / len(targets) if targets else 0
        
        overlap_score = 1.0 - max(source_overlap, target_overlap)
        
        # Pattern score (simplified)
        event_types = [e['type'] for e in group]
        pattern_score = 0.8 if len(set(event_types)) > 2 else 0.3
        
        # Combined score
        correlation_score = (time_score + overlap_score + pattern_score) / 3
        
        if correlation_score > correlation_threshold:
            correlations.append({
                'events': group,
                'score': correlation_score,
                'time_span': time_span,
                'event_types': list(set(event_types))
            })
    
    return correlations

In [ ]:
# Test event correlation
events = [
    {'timestamp': 0, 'type': 'port_scan', 'source': '192.168.1.100', 'target': '10.0.0.1'},
    {'timestamp': 5, 'type': 'login_attempt', 'source': '192.168.1.100', 'target': '10.0.0.1'},
    {'timestamp': 10, 'type': 'privilege_escalation', 'source': '192.168.1.100', 'target': '10.0.0.1'},
    {'timestamp': 100, 'type': 'data_access', 'source': '192.168.1.200', 'target': '10.0.0.2'},
    {'timestamp': 105, 'type': 'data_exfiltration', 'source': '192.168.1.200', 'target': '10.0.0.2'}
]

correlations = correlate_events(events, time_window=30)
print(f"Found {len(correlations)} correlated event groups")
for i, corr in enumerate(correlations):
    print(f"\nGroup {i+1}: Score={corr['score']:.3f}, Types={corr['event_types']}")

assert len(correlations) >= 1  # Should find at least one correlation

## Export

In [ ]:
#| export
__all__ = [
    # Threat detection
    'intrusion_detection', 'multi_stage_detector',
    
    # Anomaly detection
    'network_anomaly_detector', 'adaptive_monitor',
    
    # Intelligence and correlation
    'fuse_threat_intel', 'correlate_events'
]